# Setup

Turn on autoreload. All modules will be reloaded before excecuting each cell. Even objects that already exist will receive the new method defintions.

In [1]:
%load_ext autoreload
%autoreload 2

Import stuff.

In [2]:
import logging
import sys
import os
import atexit

sys.path.insert(0, os.path.join(os.getcwd(), ".."))
from cirrus import automate, GridSearch, LogisticRegression, graph

Configure logging so that debug-level log messages are printed.

In [3]:
log = logging.getLogger("cirrus")
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

# Done on release

Create the server image (and an instance for that purpose).

In [ ]:
instance_for_server_image = automate.Instance(
    "cirrus_instance_for_server_image",
    ami_id="ami-3a674d5a",
    region="us-west-1",
    disk_size=32,
    typ="m5.2xlarge",
    username="ec2-user"
)
instance_for_server_image.start()
automate.make_server_image(
    "cirrus_server_image",
    "s3://cirrus-public/from_shared_vm/executables",
    instance_for_server_image
)
instance_for_server_image.cleanup()

Make the lambda package.

In [ ]:
automate.make_lambda_package(
    "s3://cirrus-public/from_shared_vm/lambda-package",
    "s3://cirrus-public/from_shared_vm/executables"
)

# Done by user

Make the lambda.

In [ ]:
automate.make_lambda("cirrus_worker", "s3://cirrus-public/from_shared_vm/lambda-package")

Make the instance for the parameter servers.

In [4]:
instance_for_ps = automate.Instance(
    "cirrus_instance_for_ps",
    ami_name="cirrus_server_image",
    region="us-west-1",
    disk_size=32,
    typ="m5.2xlarge",
    username="ec2-user"
)
instance_for_ps.start()

__init__: Initializing EC2.
__init__: Resolving AMI name to AMI ID.
__init__: Done.
start: Calling _make_instance_profile.
_make_instance_profile: Initializing IAM.
_make_instance_profile: Creating role.
_make_instance_profile: Attaching policy to role.
_make_instance_profile: Creating instance profile.
_make_instance_profile: Adding role to instance profile.
_make_instance_profile: Waiting for changes to take effect.
_make_instance_profile: Done.
start: Calling _make_key_pair.
_make_key_pair: Creating new key pair.
_make_key_pair: Saving private key.
_make_key_pair: Fetching key metadata.
_make_key_pair: Done.
start: Calling _make_security_group.
_make_security_group: Creating new security group.
_make_security_group: Configuring security group.
_make_security_group: Done.
start: Calling _start_and_wait.
_start_and_wait: Starting a new instance.
_start_and_wait: Waiting for instance to enter running state.
_start_and_wait: Fetching instance metadata.
_start_and_wait: Done.
start: Done

Define the logistic regression configuration.

In [8]:
basic_params = {
    'n_workers': 10,
    'n_ps': 1,
    'lambda_size': 128,
    'dataset': "shea-criteo-kaggle-19b2",
    'learning_rate': 0.01,
    'epsilon': 0.0001,
    'progress_callback': None,
    'timeout': 0,
    'threshold_loss': 0,
    'opt_method': 'adagrad',
    'minibatch_size': 10,
    'model_bits': 19,
    'train_set': (0,6),
    'test_set': (7,8)
}

Start a grid search.

In [9]:
gs = GridSearch(
    task=LogisticRegression,
    param_base=basic_params,
    hyper_vars=["learning_rate", "lambda_size"],
    hyper_params=[[0.1, 0.2], [128, 246, 512]],
    instances=[instance_for_ps]
)
gs.set_threads(2)
gs.run(UI=True)

start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error task PID.
start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error task PID.
start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error task PID.
start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error task PID.
start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error task PID.
start: Uploading configuration.
start: Starting parameter server.
start: Retreiving parameter server PID.
start: Starting error task.
start: Retreiving error ta

View the progress of the experiment.

In [10]:
graph.display_dash()

launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda clien

launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 0, 'exp', 2, 'loss', [(14.545346, 0.6931471824645996), (25.59716, 0.6931471824645996)])
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: I

launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
('Thread', 0, 'exp', 4, 'loss', [(17.598339, 0.6931471824645996), (31.488458, 0.6931471824645996), (40.648198, 0.6931471824645996)])
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializ

('Thread', 0, 'exp', 0, 'loss', [(11.496858, 0.6931471824645996), (22.349911, 0.6931471824645996), (34.857192, 0.6931471824645996), (43.376129, 0.6931471824645996), (51.475001, 0.6931471824645996)])
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Done.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambd

launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
('Thread', 0, 'exp', 2, 'loss', [(14.545346, 0.6931471824645996), (25.59716, 0.6931471824645996), (37.798962, 0.6931471824645996), (46.007833, 0.6931471824645996), (54.321563, 0.6931471824645996), (64.942373, 0.6931471824645996)])
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
l

...

launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Inv

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 0, 'exp', 2, 'loss', [(14.545346, 0.6931471824645996), (25.59716, 0.6931471824645996), (37.798962, 0.6931471824645996), (46.007833, 0.6931471824645996), (54.321563, 0.6931471824645996), (64.942373, 0.6931471824645996), (74.705395, 0.6931471824645996), (82.577572, 0.6931471824645996), (90.971207, 0.6931471824645996), (99.152606, 0.6931471824645996), (111.447064, 0.6931471824645996), (119.752327, 0.6931471824645996), (128.253524, 0.6931471824645996), (136.679884, 0.6931471824645996), (144.988909, 0.6931471824645996), (153.622416, 0.6931471824645996), (161.940846, 0.6931471824645996), (170.058703, 0.6931471824645996), (180.381894, 0.6931471824645996), (191.201897, 0.6931471824645996), (200.052844, 0.6931471824645996), (209.959366, 0.6931

launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Done.
launch_worker: Invoking Lambda.
('Thread', 0, 'exp', 0, 'loss', [(11.496858, 0.6931471824645996), (22.349911, 0.6931471824645996), (34.857192, 0.6931471824645996), (43.376129, 0.6931471824645996), (51.475001, 0.6931471824645996), (61.076365, 0.6931471824645996), (71.903276, 0.6931471824645996), (79.894175, 0.6931471824645996), (87.868904, 0.6931471824645996), (96.18327, 0.6931471824645996), (108.597455, 0.6931471824645996), (116.892024, 0.6931471824645996), (125.613972, 0.6931471824645996), (134.025544, 0.6931471824645996), (142.260377, 0.6931471824645996), (150.781756, 0.6931471824645996), (159.104929, 0.6931471824645996), (167.324002, 0.6931471824645996), (176.287357, 0.6931471824645996), (186.467627, 0.6931471824645996), (197.30900

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_w

launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Done.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda clien

# Utilities

Write the SSH key for the instance to a file.

In [5]:
with open("key.pem", "w+") as f:
    f.write(instance_for_ps.private_key())

View the instance's public IP address.

In [6]:
instance_for_ps.public_ip()

'54.183.80.224'

In [12]:
gs.kill_all()

run_command: Running `kill $(cat error_1337.pid)`.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
run_command: Waiting for completion.
run_command: Exit code was 1.
run_command: Fetching stdout and stderr.
run_command: stdout had length 0.
run_command: stderr had length 45.
run_command: Done.
An error occurred while attempting to kill the error task. The exit code was 1 and the stderr was:
bash: line 0: kill: (2400) - No such process



RuntimeError: An error occurred while attempting to kill the error task.

launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.6931471824645996), (22.881347, 0.6931471824645996), (36.635363, 0.6931471824645996), (45.153662, 0.6931471824645996), (53.912618, 0.6931471824645996), (64.130088, 0.6931471824645996), (74.115698, 0.6931471824645996), (82.926252, 0.6931471824645996), (91.419196, 0.6931471824645996), (100.220787, 0.6931471824645996), (112.506226, 0.6931471824645996), (122.891105, 0.6931471824645996), (131.201346, 0.6931471824645996

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.

launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.6931471824645996), (22.881347, 0.6931471824645996), (36.635363, 0.6931471824645996), (45.153662, 0.6931471824645996), (53.912618, 0.6931471824645996), (64.130088, 0.6931471824645996), (74.115698, 0.6931471824645996), (82.926252, 0.6931471824645996), (91.419196, 0.6931471824645996), (100.220787, 0.6931471824645996), (112.506226, 0.6931471824645996), (122.891105, 0.6931471824645996), (131.201346, 0.6931471824645996), (139.322195, 0.6931471824645996), (147.

...

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.6931471824645996), (22.881347, 0.6931471824645996), (36.635363, 0.6931471824645996), (45.153662, 0.6931471824645996), (53.912618, 0.6931471824645996), (64.130088, 0.6931471824645996), (74.115698, 0.6931471824645996), (82.926252, 0.6931471824645996), (91.419196, 0.6931471824645996), (100.220787, 0.6931471824645996), (112.506226, 0.6931471824645996), (122.891105, 0.6931471824645996), (131.201346, 0.69314

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.6931471824645996), (22.881347, 0.6931471824645996), (36.635363, 0.6931471824645996), (45.153662,

launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
('Thread', 1, 'exp', 5, 'loss', [])
('Thread', 1, 'exp', 1, 'loss', [(11.593406, 0.6931471824645996), (22.881347, 0.6931471824645996), (36.635363, 0.6931471824645996), (45.153662,

launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda clien

launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Initializing Lambda client.
launch_worker: Invoking Lambda.
launch_worker: Done.
launch_worker: Done.
launch_worker: Init

Create a new SSH connection to the instance. Useful when the connection times out while you're aware from the computer.

In [ ]:
instance_for_ps._connect_ssh()

Do cleanup. Normally this is called on exit, but Jupyter does not reliably allow this to complete.

In [ ]:
atexit._run_exitfuncs()